# 01. Importing Libraries

In [2]:
# importing libraries
import pandas as pd
import numpy as np
import os

# 02. Import Data

In [3]:
# path shortcut
path = r'C:\Users\georg\Documents\2025-02-05 Instacart Basket Analysis'

In [5]:
# import prepared orders & products dataset
df_ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_revised.pkl'))

# 03. Grouping & Aggregating Data

## 03.01 Single Aggregation

In [6]:
# finding the average number of orders by dept - using single aggregation
df_ords_prods_merge.groupby('department_id').agg({'order_number' : ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


#### When we compare the full dataset to the subset, we can see some differences. The means are a bit different but still pretty close. For example, in the full dataset, the means for departments 1, 2, and 3 are 15.45, 17.28, and 17.17. But in the subset, those same departments have means of 14.8, 17.09, and 17.91

## 03.02 Loyalty Flag

In [7]:
# creating a new column to count the max number of orders placed by each user. 
df_ords_prods_merge['max_order'] = df_ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\georg\AppData\Local\Temp\ipykernel_20144\3271832234.py:2: FutureWarning: The provided callable <function max at 0x00000217B47D0400> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df_ords_prods_merge['max_order'] = df_ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [8]:
df_ords_prods_merge.head(15)

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,order_number,order_day_of_week,time_of_order,days_since_prior_order,product_id,add_to_cart_order,...,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,day_activity_level,busiest_period_of_day,max_order
0,0,0,2539329,1,1,2,8,NaN,196,1,...,Soda,77,7,9.0,both,Mid range product,Regularly busy,Regularly busy,Average Orders,10
1,0,0,2539329,1,1,2,8,NaN,14084,2,...,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both,Mid range product,Regularly busy,Regularly busy,Average Orders,10
2,0,0,2539329,1,1,2,8,NaN,12427,3,...,Original Beef Jerky,23,19,4.4,both,Low range product,Regularly busy,Regularly busy,Average Orders,10
3,0,0,2539329,1,1,2,8,NaN,26088,4,...,Aged White Cheddar Popcorn,23,19,4.7,both,Low range product,Regularly busy,Regularly busy,Average Orders,10
4,0,0,2539329,1,1,2,8,NaN,26405,5,...,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both,Low range product,Regularly busy,Regularly busy,Average Orders,10
5,1,1,2398795,1,2,3,7,15.0,196,1,...,Soda,77,7,9.0,both,Mid range product,Regularly busy,Least busiest days,Average Orders,10
6,1,1,2398795,1,2,3,7,15.0,10258,2,...,Pistachios,117,19,3.0,both,Low range product,Regularly busy,Least busiest days,Average Orders,10
7,1,1,2398795,1,2,3,7,15.0,12427,3,...,Original Beef Jerky,23,19,4.4,both,Low range product,Regularly busy,Least busiest days,Average Orders,10
8,1,1,2398795,1,2,3,7,15.0,13176,4,...,Bag of Organic Bananas,24,4,10.3,both,Mid range product,Regularly busy,Least busiest days,Average Orders,10
9,1,1,2398795,1,2,3,7,15.0,26088,5,...,Aged White Cheddar Popcorn,23,19,4.7,both,Low range product,Regularly busy,Least busiest days,Average Orders,10


In [9]:
# Assigning Loyalty Flag
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [10]:
df_ords_prods_merge.loc[(df_ords_prods_merge['max_order'] <= 40) & (df_ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [13]:
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [14]:
df_ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

In [18]:
df_summary = df_ords_prods_merge.groupby('loyalty_flag').agg({'prices' : ['sum', 'mean', 'min', 'max']})

In [19]:
df_summary

prices                         
                          sum       mean  min      max
loyalty_flag                                          
Loyal customer    106814042.2  10.386336  1.0  99999.0
New customer       83011787.2  13.294670  1.0  99999.0
Regular customer  198391693.2  12.495717  1.0  99999.0

#### Not too sure about the min & max prices. 
#### We can tell that in total, the regular customers spent the most - almost double what teh loyal customers spent (198M vs 106M) - while new customers spent the least at 83M
#### New customers on the other hand that the largest average amount spent per order, coming out to 13.3 . Loyal customers averaged the least with 10.4 and Regular customers spent 12.5 on average

## 03.03 Spending Flag

In [26]:
# creating a new column to calculate the average amount spent by each user id. 
df_ords_prods_merge['avg_product_price'] = df_ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\georg\AppData\Local\Temp\ipykernel_20144\3846347510.py:2: FutureWarning: The provided callable <function mean at 0x00000217B47D0E00> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df_ords_prods_merge['avg_product_price'] = df_ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [29]:
df_ords_prods_merge.head(15)

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,order_number,order_day_of_week,time_of_order,days_since_prior_order,product_id,add_to_cart_order,...,department_id,prices,_merge,price_range_loc,busiest_day,day_activity_level,busiest_period_of_day,max_order,loyalty_flag,avg_product_price
0,0,0,2539329,1,1,2,8,NaN,196,1,...,7,9.0,both,Mid range product,Regularly busy,Regularly busy,Average Orders,10,New customer,6.367797
1,0,0,2539329,1,1,2,8,NaN,14084,2,...,16,12.5,both,Mid range product,Regularly busy,Regularly busy,Average Orders,10,New customer,6.367797
2,0,0,2539329,1,1,2,8,NaN,12427,3,...,19,4.4,both,Low range product,Regularly busy,Regularly busy,Average Orders,10,New customer,6.367797
3,0,0,2539329,1,1,2,8,NaN,26088,4,...,19,4.7,both,Low range product,Regularly busy,Regularly busy,Average Orders,10,New customer,6.367797
4,0,0,2539329,1,1,2,8,NaN,26405,5,...,17,1.0,both,Low range product,Regularly busy,Regularly busy,Average Orders,10,New customer,6.367797
5,1,1,2398795,1,2,3,7,15.0,196,1,...,7,9.0,both,Mid range product,Regularly busy,Least busiest days,Average Orders,10,New customer,6.367797
6,1,1,2398795,1,2,3,7,15.0,10258,2,...,19,3.0,both,Low range product,Regularly busy,Least busiest days,Average Orders,10,New customer,6.367797
7,1,1,2398795,1,2,3,7,15.0,12427,3,...,19,4.4,both,Low range product,Regularly busy,Least busiest days,Average Orders,10,New customer,6.367797
8,1,1,2398795,1,2,3,7,15.0,13176,4,...,4,10.3,both,Mid range product,Regularly busy,Least busiest days,Average Orders,10,New customer,6.367797
9,1,1,2398795,1,2,3,7,15.0,26088,5,...,19,4.7,both,Low range product,Regularly busy,Least busiest days,Average Orders,10,New customer,6.367797


In [31]:
# Assigning spending flag
df_ords_prods_merge.loc[df_ords_prods_merge['avg_product_price'] < 10, 'spending_flag'] = 'Low Spender'

In [32]:
df_ords_prods_merge.loc[df_ords_prods_merge['avg_product_price'] >= 10, 'spending_flag'] = 'High Spender'

In [33]:
df_ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low Spender     31770614
High Spender      634245
Name: count, dtype: int64

## 03.04 Order Frequency Flag

In [34]:
# creating a new column to calculate the average amount spent by each user id. 
df_ords_prods_merge['medain_days_since_last_order'] = df_ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

C:\Users\georg\AppData\Local\Temp\ipykernel_20144\237626874.py:2: FutureWarning: The provided callable <function median at 0x00000217B492BA60> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  df_ords_prods_merge['medain_days_since_last_order'] = df_ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [35]:
df_ords_prods_merge.head(15)

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,order_number,order_day_of_week,time_of_order,days_since_prior_order,product_id,add_to_cart_order,...,_merge,price_range_loc,busiest_day,day_activity_level,busiest_period_of_day,max_order,loyalty_flag,avg_product_price,spending_flag,medain_days_since_last_order
0,0,0,2539329,1,1,2,8,NaN,196,1,...,both,Mid range product,Regularly busy,Regularly busy,Average Orders,10,New customer,6.367797,Low Spender,20.5
1,0,0,2539329,1,1,2,8,NaN,14084,2,...,both,Mid range product,Regularly busy,Regularly busy,Average Orders,10,New customer,6.367797,Low Spender,20.5
2,0,0,2539329,1,1,2,8,NaN,12427,3,...,both,Low range product,Regularly busy,Regularly busy,Average Orders,10,New customer,6.367797,Low Spender,20.5
3,0,0,2539329,1,1,2,8,NaN,26088,4,...,both,Low range product,Regularly busy,Regularly busy,Average Orders,10,New customer,6.367797,Low Spender,20.5
4,0,0,2539329,1,1,2,8,NaN,26405,5,...,both,Low range product,Regularly busy,Regularly busy,Average Orders,10,New customer,6.367797,Low Spender,20.5
5,1,1,2398795,1,2,3,7,15.0,196,1,...,both,Mid range product,Regularly busy,Least busiest days,Average Orders,10,New customer,6.367797,Low Spender,20.5
6,1,1,2398795,1,2,3,7,15.0,10258,2,...,both,Low range product,Regularly busy,Least busiest days,Average Orders,10,New customer,6.367797,Low Spender,20.5
7,1,1,2398795,1,2,3,7,15.0,12427,3,...,both,Low range product,Regularly busy,Least busiest days,Average Orders,10,New customer,6.367797,Low Spender,20.5
8,1,1,2398795,1,2,3,7,15.0,13176,4,...,both,Mid range product,Regularly busy,Least busiest days,Average Orders,10,New customer,6.367797,Low Spender,20.5
9,1,1,2398795,1,2,3,7,15.0,26088,5,...,both,Low range product,Regularly busy,Least busiest days,Average Orders,10,New customer,6.367797,Low Spender,20.5


In [36]:
# Assigning order freq flag
df_ords_prods_merge.loc[df_ords_prods_merge['medain_days_since_last_order'] > 20, 'order_frequency_flag'] = 'Non-Frequent Customer'

In [37]:
df_ords_prods_merge.loc[(df_ords_prods_merge['medain_days_since_last_order'] > 10) & (df_ords_prods_merge['medain_days_since_last_order'] <= 20), 'order_frequency_flag'] = 'Regular Customer'

In [38]:
df_ords_prods_merge.loc[df_ords_prods_merge['medain_days_since_last_order'] <= 10, 'order_frequency_flag'] = 'Frequent Customer'

In [39]:
df_ords_prods_merge['order_frequency_flag'].value_counts(dropna = False)

order_frequency_flag
Frequent Customer        21559853
Regular Customer          7208564
Non-Frequent Customer     3636437
NaN                             5
Name: count, dtype: int64

# 04. Exporting Data

In [41]:
df_ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_grouped.pkl'))